In [5]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2     #opencv-python, 컴퓨터비전 관련 tool
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary    #torchinfo
from torchvision import transforms
from torchvision.models import resnet50

In [6]:
#커스텀 데이터셋 만들기
class Dataset(Dataset):
    def __init__(self, dir, image_ids, transforms):
        self.dir = dir
        self.transforms = transforms
        
        self.labels = []
        with open(image_ids, 'r') as f:
            reader = csv.reader()
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))
        self.image_ids = list(self.labels.keys())
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, index):
        image_id = self.image_ids[index]
        image = Image.open(os.path.join(self.dir, f'{str(image_id).zfill(5)}png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)
        
        if self.transforms is not None:
            image = self.transforms(image)
            
        return image, target

In [7]:
#데이터 augumentation
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])     #[평균],[분산]
])    #RandomHorizontalFlip.. 뿐만 아니라 rotation, cropping 등 다른 augumentation방법도 사용가능
transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [16]:
#사용자 데이터셋 불러오기
trainset = Dataset('data/train', 'data/train.csv', transforms_train)
testset = Dataset('data/test', 'data/test.csv', transforms_test)

train_loader(DataLoader(trainset, batch_size = 256, num_workers=0))
test_loader(DataLoader(testset, batch_size = 32, num_workers=0))

TypeError:  expected at least 1 argument, got 0

In [12]:
#pretrained된 resnet50 네트워크 불러오기

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet=(resnet50(pretrained=True))
        self.classifier = nn.Linear(1000,26)
        
    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)
        
        return x

#GPU가속화를 위한 코드
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Model().to(device)
print(summary(model, input_size=(1,3,256,256), verbose = 0))
# .pth : 저장된 모델의 확장자

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\03/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
24.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

70.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%


Layer (type:depth-idx)                        Output Shape              Param #
Model                                         --                        --
├─ResNet: 1-1                                 [1, 1000]                 --
│    └─Conv2d: 2-1                            [1, 64, 128, 128]         9,408
│    └─BatchNorm2d: 2-2                       [1, 64, 128, 128]         128
│    └─ReLU: 2-3                              [1, 64, 128, 128]         --
│    └─MaxPool2d: 2-4                         [1, 64, 64, 64]           --
│    └─Sequential: 2-5                        [1, 256, 64, 64]          --
│    │    └─Bottleneck: 3-1                   [1, 256, 64, 64]          75,008
│    │    └─Bottleneck: 3-2                   [1, 256, 64, 64]          70,400
│    │    └─Bottleneck: 3-3                   [1, 256, 64, 64]          70,400
│    └─Sequential: 2-6                        [1, 512, 32, 32]          --
│    │    └─Bottleneck: 3-4                   [1, 512, 32, 32]          379,392

In [14]:
#학습을 위한 코드
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 100
model.train()

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        
        images = images.to(device)
        targets = targets.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            outputs = outputs>0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch}:{loss.item():.5f},{acc.item():.5f}')

NameError: name 'train_loader' is not defined

In [ ]:
#사용해보지 않았던 csv파일을 사용, pretrained된 모델 불러오는 방법